<a href="https://colab.research.google.com/github/AyeshaNayyer/Sign-Language-recognition-using-Mediapipe/blob/main/training_task_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install mediapipe-model-maker

In [ ]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')
from mediapipe_model_maker import gesture_recognizer
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "mallickhiba"
os.environ['KAGGLE_KEY'] = "be6f1f084ef0f6ceec205351e86bfacb"


In [ ]:
!kaggle datasets download -d grassknoted/asl-alphabet

 99% 1.01G/1.03G [00:08<00:00, 132MB/s]
100% 1.03G/1.03G [00:08<00:00, 128MB/s]


In [ ]:
!kaggle datasets download -d danrasband/asl-alphabet-test

 74% 18.0M/24.3M [00:00<00:00, 79.9MB/s]
100% 24.3M/24.3M [00:00<00:00, 66.3MB/s]


In [ ]:
!unzip /content/asl-alphabet.zip
!unzip /content/asl-alphabet-test.zip

Streaming output truncated to the last 5000 lines.
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing766.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing767.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing768.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing769.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing77.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing770.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing771.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing772.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing773.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing774.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing775.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing776.jpg  
  inflating: asl_alphabet_train/asl_al

In [ ]:
dataset_path = "/content/asl_alphabet_train/asl_alphabet_train"

In [ ]:
#!wget /train.zip
#!unzip /train.zip
#dataset_path = "train"

In [ ]:
print(dataset_path)
labels = []
for i in os.listdir(dataset_path):
  if os.path.isdir(os.path.join(dataset_path, i)):
    labels.append(i)

print(labels)

/content/asl_alphabet_train/asl_alphabet_train
['Y', 'space', 'Q', 'Z', 'D', 'M', 'C', 'I', 'V', 'U', 'B', 'A', 'L', 'F', 'nothing', 'E', 'J', 'W', 'T', 'G', 'P', 'del', 'R', 'O', 'S', 'K', 'X', 'N', 'H']


In [ ]:
NUM_EXAMPLES = 5

for label in labels:
  label_dir = os.path.join(dataset_path, label)
  example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))
  for i in range(NUM_EXAMPLES):
    axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
    axs[i].get_xaxis().set_visible(False)
    axs[i].get_yaxis().set_visible(False)
  fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')

plt.show()

In [ ]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

In [ ]:

hparams = gesture_recognizer.HParams(learning_rate=0.001,epochs= 25,batch_size= 128, shuffle = True, lr_decay = 0.99, export_dir="exported_model")
model_options = gesture_recognizer.ModelOptions(dropout_rate=0.2)
options = gesture_recognizer.GestureRecognizerOptions(model_options=model_options, hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)


In [ ]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")


In [ ]:
model.export_model()
!ls exported_model

In [ ]:
files.download('exported_model/gesture_recognizer.task')